# Sentiment classification

In [1]:
!pip install torch==1.11.*
!pip install torchvision==0.12.*
!pip install fairscale==0.4.*
!pip install einops==0.4.*
!pip install tokenizers==0.12.*
!pip install jsonargparse==4.7.*
!pip install pytorch-lightning==1.6.*

     |████████████████████████████████| 248 kB 29.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for fairscale: filename=fairscale-0.4.6-py3-none-any.whl size=307252 sha256=f5141ba649fa0f6febe387c39256287a12bd53bc77d3315969c961a8c90b49ef
  Stored in directory: /root/.cache/pip/wheels/4e/4f/0b/94c29ea06dfad93260cb0377855f87b7b863312317a7f69fe7
Successfully built fairscale
     |████████████████████████████████| 6.6 MB 27.0 MB/s 
     |████████████████████████████████| 135 kB 23.5 MB/s 
     |████████████████████████████████| 582 kB 30.1 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
     |████████████████████████████████| 136 kB 56.8 MB/s 
     |████████████████████████████████| 409 kB 51.8 MB/s 
     |████████████████████████████████| 1.1 MB 7.0 MB/s 
     |████████████████████████████████| 271 kB 47.0 MB/s 
     |███████████

In [2]:
!git clone https://github.com/krasserm/perceiver-io.git -b wip-enhancements

Cloning into 'perceiver-io'...
remote: Enumerating objects: 550, done.
remote: Counting objects: 100% (241/241), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 550 (delta 148), reused 153 (delta 106), pack-reused 309
Receiving objects: 100% (550/550), 951.52 KiB | 25.04 MiB/s, done.
Resolving deltas: 100% (310/310), done.


In [3]:
# Download model checkpoints
!wget -nc -O logs.zip https://martin-krasser.com/perceiver/logs-update-3.zip
!unzip -qo logs.zip

--2022-05-09 13:42:51--  https://martin-krasser.com/perceiver/logs-update-3.zip
Resolving martin-krasser.com (martin-krasser.com)... 217.160.0.142, 2001:8d8:100f:f000::209
Connecting to martin-krasser.com (martin-krasser.com)|217.160.0.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44295508 (42M) [application/zip]
Saving to: ‘logs.zip’

logs.zip            100%[===================>]  42.24M  21.2MB/s    in 2.0s    

2022-05-09 13:42:53 (21.2 MB/s) - ‘logs.zip’ saved [44295508/44295508]



In [4]:
import sys
import torch

sys.path.append("perceiver-io")

from perceiver.data.text_preproc import TextPreprocessor
from perceiver.model.lightning import LitTextClassifier

In [5]:
imdb_preproc = TextPreprocessor(tokenizer_path='perceiver-io/.cache/imdb-tokenizer-10003.json')

In [6]:
ckpt_path = 'logs/seq_clf/version_1/checkpoints/epoch=035-val_loss=0.259.ckpt'

model = LitTextClassifier.load_from_checkpoint(ckpt_path, clf_ckpt=None).model
model.eval();

In [7]:
text_batch = [
    "I've seen this movie yesterday and it was really boring",
    "I can recommend this movie to all fantasy movie lovers"
]

In [8]:
with torch.no_grad():
    logits = model(*imdb_preproc.preprocess_batch(text_batch))
    preds = logits.argmax(dim=1)

In [9]:
for text, pred in zip(text_batch, preds):
    print(f'{text} (positive = {pred == 1})')

I've seen this movie yesterday and it was really boring (positive = False)
I can recommend this movie to all fantasy movie lovers (positive = True)
